In [40]:
import numpy as np
from sentence_transformers import SentenceTransformer
from text_untils import *
import pickle
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
with open('output/bm25/bm25_65k_questions_new/bm25plus.pkl', 'rb') as f:
    bm25plus = pickle.load(f)

In [42]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/bm25_65k_questions_new/docs.jsonl')

In [43]:
query = "Đi xe máy vi phạm nồng độ cồn bị phạt bao nhiêu tiền?"
query = clean_text(query)
query = word_segment(query)
query = remove_stopword(normalize_text(query))

query_bm25 = query.split()
print(query_bm25)

['đi', 'xe_máy', 'vi_phạm', 'nồng_độ', 'cồn', 'phạt', 'bao_nhiêu', 'tiền']


# **BM25**

In [44]:
bm25_scores = bm25plus.get_scores(query_bm25)
top_n_indices = np.argsort(bm25_scores)[::-1][:50]
top_n_score = bm25_scores[top_n_indices]

topn_docs_bm25 = [docs[idx] for idx in top_n_indices]

In [45]:
for doc_dict, score in zip(topn_docs_bm25, top_n_score):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Score: {score}")

ID: 6663569fc2f544363eec84dc, Document: đi xe_máy vi_phạm nồng_độ cồn có_thể phạt bao_nhiêu tiền, Score: 65.59116414126893
ID: 6663569fc2f544363eec84dd, Document: đi xe_máy vi_phạm nồng_độ cồn nộp phạt, Score: 64.08573026612979
ID: 6663569fc2f544363eec84de, Document: đi xe_máy ký biên_bản phạt vi_phạm nồng_độ cồn phạt, Score: 62.16723513997646
ID: 6663569ec2f544363eec719e, Document: điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền, Score: 62.09109636324652
ID: 6663569dc2f544363eec60d6, Document: phạt nồng_độ cồn xe_máy bao_nhiêu, Score: 61.92960124677894
ID: 6663569dc2f544363eec5f6b, Document: vi_phạm nồng_độ cồn tạm xe_máy bao_nhiêu, Score: 60.84051108308686
ID: 6663569dc2f544363eec5f6a, Document: phạt nồng_độ cồn xe_máy 2024 bao_nhiêu, Score: 60.09489709656561
ID: 6663568cc2f544363eec07a4, Document: phạt nồng_độ cồn xe_máy 2024 bao_nhiêu, Score: 60.09489709656561
ID: 6663569ac2f544363eec2d7d, Document: vi_phạm nồng_độ cồn xe_máy bao_lâu, Score: 60.01817518251619
I

In [46]:
max_score = max(top_n_score)
rescaled_scores = [1*score/max_score for score in top_n_score]

final_top_docs_bm25 = []
for (doc_dict, score) in zip(topn_docs_bm25, rescaled_scores):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Rescaled Score: {score:.4f}")
    final_top_docs_bm25.append({'id': doc_id, 'title': doc_title, 'bm25_score': score})

ID: 6663569fc2f544363eec84dc, Document: đi xe_máy vi_phạm nồng_độ cồn có_thể phạt bao_nhiêu tiền, Rescaled Score: 1.0000
ID: 6663569fc2f544363eec84dd, Document: đi xe_máy vi_phạm nồng_độ cồn nộp phạt, Rescaled Score: 0.9770
ID: 6663569fc2f544363eec84de, Document: đi xe_máy ký biên_bản phạt vi_phạm nồng_độ cồn phạt, Rescaled Score: 0.9478
ID: 6663569ec2f544363eec719e, Document: điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền, Rescaled Score: 0.9466
ID: 6663569dc2f544363eec60d6, Document: phạt nồng_độ cồn xe_máy bao_nhiêu, Rescaled Score: 0.9442
ID: 6663569dc2f544363eec5f6b, Document: vi_phạm nồng_độ cồn tạm xe_máy bao_nhiêu, Rescaled Score: 0.9276
ID: 6663569dc2f544363eec5f6a, Document: phạt nồng_độ cồn xe_máy 2024 bao_nhiêu, Rescaled Score: 0.9162
ID: 6663568cc2f544363eec07a4, Document: phạt nồng_độ cồn xe_máy 2024 bao_nhiêu, Rescaled Score: 0.9162
ID: 6663569ac2f544363eec2d7d, Document: vi_phạm nồng_độ cồn xe_máy bao_lâu, Rescaled Score: 0.9150
ID: 6663569dc2f5443

# **bi-encoder**

In [47]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

In [48]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [49]:
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
model_bi_encoder = AutoModel.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

In [50]:
index = faiss.read_index('output/sbert/embeddings_65k_index.index')

encoded_query = tokenizer([query], padding=True, truncation=True, return_tensors='pt', max_length=256)

with torch.no_grad():
    model_output_query = model_bi_encoder(**encoded_query)

embeddings_query = mean_pooling(model_output_query, encoded_query['attention_mask'])
embeddings_query = embeddings_query.cpu().numpy()
faiss.normalize_L2(embeddings_query)

In [51]:
k = 50
distances_bi_encoder, indices = index.search(embeddings_query, k)

In [52]:
topn_doc_bi_encoder = [docs[idx] for idx in indices[0]]

In [53]:
final_top_docs_bi_encoder = []
for distance, doc_dict in zip(distances_bi_encoder[0], topn_doc_bi_encoder):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"{doc_id}, Document: {doc_title}, Distance: {distance:.4f}")
    final_top_docs_bi_encoder.append({'id': doc_id, 'title': doc_title, 'bi_encoder_score': distance})

6663569ec2f544363eec719e, Document: điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền, Distance: 0.9253
6663569fc2f544363eec817c, Document: lái_xe_máy nồng_độ cồn bao_nhiêu phạt, Distance: 0.8950
6663569cc2f544363eec42d8, Document: lỗi vi_phạm nồng_độ cồn điều_khiển xe_máy tham_gia giao_thông phạt bao_nhiêu tiền, Distance: 0.8805
6663569ac2f544363eec2d7c, Document: lái xe_máy vi_phạm lỗi nồng_độ cồn tạm xe_máy, Distance: 0.8306
6663568cc2f544363eec06a4, Document: điều_khiển xe_máy sử_dụng rượu_bia tham_gia giao_thông phạt bao_nhiêu tiền, Distance: 0.8256
6663569bc2f544363eec3ccf, Document: lỗi đi xe_máy đèn_đỏ phạt bao_nhiêu tiền, Distance: 0.8003
6663569ec2f544363eec715f, Document: điều_khiển xe_máy đèn_đỏ phạt bao_nhiêu tiền, Distance: 0.7882
6663569fc2f544363eec7bb6, Document: đi xe_máy đèn_đỏ phạt bao_nhiêu tiền, Distance: 0.7694
6663569dc2f544363eec60d6, Document: phạt nồng_độ cồn xe_máy bao_nhiêu, Distance: 0.7687
6663569dc2f544363eec60d5, Document: nồng_độ cồn

### Combined Bi-encoder and BM25

In [54]:
final_ensemble = {}

In [55]:
from collections import defaultdict
combined_retrieval_scores = defaultdict(lambda: {'bm25_score': 0, 'bi_encoder_score': 0})

In [56]:
for doc in final_top_docs_bm25:
    combined_retrieval_scores[doc['id']]['bm25_score'] = doc['bm25_score']

In [57]:
for doc in final_top_docs_bi_encoder:
    combined_retrieval_scores[doc['id']]['bi_encoder_score'] = doc['bi_encoder_score']

In [58]:
import math
final_retrieval_scores = []
for doc_id, scores in combined_retrieval_scores.items():
    if scores['bm25_score'] > 0 and scores['bi_encoder_score'] > 0:
        combined_score = math.sqrt(scores['bm25_score'] * scores['bi_encoder_score'])
        final_retrieval_scores.append({'id': doc_id, 'retrieval_score': combined_score})

        if doc_id not in final_ensemble:
            final_ensemble[doc_id] = {'id': doc_id}
        final_ensemble[doc_id]['bm25_score'] = scores['bm25_score']

        if doc_id not in final_ensemble:
            final_ensemble[doc_id] = {'id': doc_id}
        final_ensemble[doc_id]['bi_encoder_score'] = scores['bi_encoder_score']

In [59]:
final_scores_retrieval = sorted(final_retrieval_scores, key=lambda x: x['retrieval_score'], reverse=True)
top_20_scores_retrieval = final_scores_retrieval[:20]

In [60]:
for doc in top_20_scores_retrieval:
    print(f"ID: {doc['id']},  Combined Score: {doc['retrieval_score']:.4f}")

ID: 6663569ec2f544363eec719e,  Combined Score: 0.9359
ID: 6663569cc2f544363eec42d8,  Combined Score: 0.8924
ID: 6663569fc2f544363eec817c,  Combined Score: 0.8739
ID: 6663569fc2f544363eec84dc,  Combined Score: 0.8615
ID: 6663569ac2f544363eec2d7c,  Combined Score: 0.8546
ID: 6663569dc2f544363eec60d6,  Combined Score: 0.8519
ID: 6663569dc2f544363eec60d5,  Combined Score: 0.8381
ID: 6663561bc2f544363eea95c7,  Combined Score: 0.8341
ID: 6663569dc2f544363eec5f6b,  Combined Score: 0.8237
ID: 6663569ac2f544363eec2d7d,  Combined Score: 0.8187
ID: 6663569fc2f544363eec817d,  Combined Score: 0.7984
ID: 6663569dc2f544363eec5f6a,  Combined Score: 0.7981
ID: 6663569fc2f544363eec841d,  Combined Score: 0.7959
ID: 6663569fc2f544363eec7bb6,  Combined Score: 0.7914
ID: 6663568cc2f544363eec07a5,  Combined Score: 0.7849
ID: 6663569ec2f544363eec6808,  Combined Score: 0.7673
ID: 6663569dc2f544363eec513d,  Combined Score: 0.7673
ID: 66635739c2f544363eee90b5,  Combined Score: 0.7656
ID: 6663569bc2f544363eec3692

# **cross-encoder**

In [61]:
from sentence_transformers import CrossEncoder
model_cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512, default_activation_function=torch.nn.Sigmoid(), device='cuda')

In [62]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd

def get_selected_rows_from_mongodb(database_name, collection_name, top_docs):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database_name]
    collection = db[collection_name]

    doc_ids = [ObjectId(doc_dict['id']) for doc_dict in top_docs]

    cursor = collection.find({'_id': {'$in': doc_ids}}, {
        'title': 1,
        'reference': 1,
        'quote': 1,
        'conclusion': 1
    })
    data = list(cursor)
    df = pd.DataFrame(data)

    # client.close()
    return df

In [63]:
selected_rows = get_selected_rows_from_mongodb('lawlaboratory', 'questions_cleaned', top_20_scores_retrieval)

In [64]:
selected_rows

,_id,title,conclusion,reference,quote
0,6663561bc2f544363eea95c7,Giao xe máy cho người có nồng độ cồn trong ngư...,[Nếu bạn giao xe máy của bạn cho đồng nghiệp n...,Tại khoản 5 Điều 30 Nghị định 100/2019/NĐ-CP đ...,{'name': 'đ) Giao xe hoặc để cho người không đ...
1,6663568cc2f544363eec07a5,Lỗi nồng độ cồn xe máy có bị giam xe không?,"[Theo đó, người điều khiển xe trên đường mà tr...",Căn cứ khoản 1 Điều 82 Nghị định 100/2019/NĐ-C...,"{'name': 'b) Điểm b, điểm c khoản 6; điểm c kh..."
2,6663569ac2f544363eec2d7c,Người lái xe máy vi phạm lỗi nồng độ cồn có bị...,"[Với quy định này, người lái xe máy vi phạm lỗ...",Căn cứ theo khoản 1 Điều 82 Nghị định 100/2019...,"{'name': 'Tạm giữ phương tiện, giấy tờ có liên..."
3,6663569ac2f544363eec2d7d,Người vi phạm nồng độ cồn bị giữ xe máy trong ...,"[Theo đó, thời hạn tạm giữ xe vi phạm nồng độ ...",Căn cứ theo khoản 8 Điều 125 Luật Xử lý vi phạ...,"{'name': 'Tạm giữ tang vật, phương tiện, giấy ..."
4,6663569bc2f544363eec3692,Mức phạt với hành vi vi phạm nồng độ cồn của x...,"[Như vậy, mức phạt đối với người điều khiển xe...","Căn cứ tại điểm q khoản 1, điểm e khoản 3, điể...","{'name': 'Xử phạt người điều khiển xe đạp, xe ..."
5,6663569cc2f544363eec42d8,Lỗi vi phạm về nồng độ cồn khi điều khiển xe m...,"[Theo đó, khi xử phạt sẽ dựa trên nồng độ cồn ...","Căn cứ theo khoản 6, 7, 8 Điều 6 Nghị định 100...","{'name': 'Xử phạt người điều khiển xe mô tô, x..."
6,6663569dc2f544363eec513d,Mức phạt nồng độ cồn đối với người điều khiển ...,"[Như vậy, người điều khiển xe máy mà trong máu...",Căn cứ điểm c khoản 6; điểm c khoản 7; điểm e ...,"{'name': 'Xử phạt người điều khiển xe mô tô, x..."
7,6663569dc2f544363eec5f6a,Mức phạt nồng độ cồn xe máy năm 2024 là bao nh...,"[Như vậy, mức phạt nồng độ cồn hiện nay được q...",Tại khoản 6 Điều 6 Nghị định 100/2019/NĐ-CP sử...,"{'name': 'Xử phạt người điều khiển xe mô tô, x..."
8,6663569dc2f544363eec5f6b,Vi phạm nồng độ cồn bị tạm giữ xe máy bao nhiê...,"[Như vậy, thời hạn tạm giữ xe đối với hành vi ...",Tại khoản 1 Điều 82 Nghị định 100/2019/NĐ-CP đ...,"{'name': 'Tạm giữ tang vật, phương tiện, giấy ..."
9,6663569dc2f544363eec60d5,Nồng độ cồn bao nhiêu thì bị giữ xe máy?,"[Theo đó, người điều khiển có thể bị tạm giữ x...",Tại Điều 82 Nghị định 100/2019/NĐ-CP được sửa ...,"{'name': 'Xử phạt người điều khiển xe mô tô, x..."


In [65]:
def merge_quote(quote):
    if isinstance(quote, dict):
        name = quote.get('name', ' ')
        content = ' '.join(quote.get('content', []))

        if name is None:
            name = ' '
        if content is None:
            content = ' '

        return f"{name}. {content}"
    else:
        return None

selected_rows['conclusion'] = selected_rows.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)

In [66]:
def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [67]:
def create_answer(row):
    conclusion_value = row['conclusion']

    if isinstance(conclusion_value, list):
        conclusion_value = ' '.join(conclusion_value)
    elif pd.isna(conclusion_value):
        conclusion_value = ''

    quote_value= ''
    if not pd.isna(row['quote']):
        reference = row['reference'] if not pd.isna(row['reference']) else ''
        merged_quote = merge_quote(row['quote'])
        quote_value = f"{reference} {merged_quote}"

    conclusion_tokens = count_tokens(conclusion_value)
    quote_tokens = count_tokens(quote_value)

    if conclusion_tokens >= quote_tokens:
        return conclusion_value
    else:
        return quote_value

selected_rows['answer'] = selected_rows.apply(create_answer, axis=1)

In [68]:
answers = selected_rows['answer'].tolist()

In [69]:
query_answer_pairs = [(query, answer) for answer in answers]

In [70]:
scores = model_cross_encoder.predict(query_answer_pairs, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
final_top_docs_cross_encoder = []
for index, row in selected_rows.iterrows():
    doc_id = row['_id']
    title = row['title']
    score = scores[index]
    print(f"ID: {doc_id}, Title: {title}, Answer_Score: {score:.4f}")
    final_top_docs_cross_encoder.append({'id': doc_id, 'title': title, 'cross_encoder_score': score})

ID: 6663561bc2f544363eea95c7, Title: Giao xe máy cho người có nồng độ cồn trong người điều khiển bị phạt bao nhiêu tiền?, Answer_Score: 0.9837
ID: 6663568cc2f544363eec07a5, Title: Lỗi nồng độ cồn xe máy có bị giam xe không?, Answer_Score: 0.5825
ID: 6663569ac2f544363eec2d7c, Title: Người lái xe máy vi phạm lỗi nồng độ cồn có bị tạm giữ xe máy không?, Answer_Score: 0.7106
ID: 6663569ac2f544363eec2d7d, Title: Người vi phạm nồng độ cồn bị giữ xe máy trong bao lâu?, Answer_Score: 0.8239
ID: 6663569bc2f544363eec3692, Title: Mức phạt với hành vi vi phạm nồng độ cồn của xe đạp điện là bao nhiêu?, Answer_Score: 0.9919
ID: 6663569cc2f544363eec42d8, Title: Lỗi vi phạm về nồng độ cồn khi điều khiển xe máy tham gia giao thông phạt bao nhiêu tiền?, Answer_Score: 0.9949
ID: 6663569dc2f544363eec513d, Title: Mức phạt nồng độ cồn đối với người điều khiển xe máy năm 2024 là bao nhiêu?, Answer_Score: 0.9920
ID: 6663569dc2f544363eec5f6a, Title: Mức phạt nồng độ cồn xe máy năm 2024 là bao nhiêu?, Answer_Sc

# Ensemble

In [72]:
for record in final_top_docs_cross_encoder:
    doc_id = str(record['id'])
    if doc_id not in final_ensemble:
        final_ensemble[doc_id] = {'id': doc_id}
    final_ensemble[doc_id]['cross_encoder_score'] = record['cross_encoder_score']

In [73]:
alpha = 0.6
def calculate_ensemble_score(bm25_score, bi_encoder_score, cross_encoder_score):
    return (1-alpha) * bm25_score + alpha * math.sqrt(bi_encoder_score * cross_encoder_score)

In [74]:
final_combined_scores = {}

In [75]:
for doc_id, scores in final_ensemble.items():
    cross_encoder_score = scores.get('cross_encoder_score', None)
    bm25_score = scores.get('bm25_score', None)
    bi_encoder_score = scores.get('bi_encoder_score', None)
    print(f"ID: {doc_id}, BM25: {bm25_score}, Bi-encoder: {bi_encoder_score}, Cross-encoder: {cross_encoder_score}")

    if cross_encoder_score is not None and bm25_score is not None and bi_encoder_score is not None:
        final_combined_scores[doc_id] = calculate_ensemble_score(bm25_score, bi_encoder_score, cross_encoder_score)

ID: 6663569fc2f544363eec84dc, BM25: 1.0, Bi-encoder: 0.7421891689300537, Cross-encoder: 0.9909096360206604
ID: 6663569ec2f544363eec719e, BM25: 0.9466381207919402, Bi-encoder: 0.9253069162368774, Cross-encoder: 0.9964761137962341
ID: 6663569dc2f544363eec60d6, BM25: 0.9441759733581829, Bi-encoder: 0.7686600685119629, Cross-encoder: 0.9974090456962585
ID: 6663569dc2f544363eec5f6b, BM25: 0.9275717526837881, Bi-encoder: 0.7314245104789734, Cross-encoder: 0.6107956767082214
ID: 6663569dc2f544363eec5f6a, BM25: 0.9162041546805668, Bi-encoder: 0.6951665878295898, Cross-encoder: 0.9977487921714783
ID: 6663569ac2f544363eec2d7d, BM25: 0.9150344557576421, Bi-encoder: 0.732483983039856, Cross-encoder: 0.8238816261291504
ID: 6663569dc2f544363eec60d5, BM25: 0.9139178959855253, Bi-encoder: 0.7686067819595337, Cross-encoder: 0.9959509372711182
ID: 6663569fc2f544363eec841d, BM25: 0.9137198065222669, Bi-encoder: 0.6933384537696838, Cross-encoder: 0.9933494925498962
ID: 6663561bc2f544363eea95c7, BM25: 0.91

In [78]:
final_combined_scores = sorted(final_combined_scores.items(), key=lambda x: x[1], reverse=True)

In [79]:
final_combined_scores

[('6663569ec2f544363eec719e', 0.9547946827352146),
 ('6663569cc2f544363eec42d8', 0.9233222365926965),
 ('6663569fc2f544363eec84dc', 0.9145476304408656),
 ('6663569fc2f544363eec817c', 0.9077356832613825),
 ('6663569dc2f544363eec60d6', 0.903028038441285),
 ('6663569dc2f544363eec60d5', 0.8905224632260362),
 ('6663561bc2f544363eea95c7', 0.8847424848531726),
 ('6663569dc2f544363eec5f6a', 0.8661781487314923),
 ('6663569fc2f544363eec841d', 0.8634255255113963),
 ('6663569dc2f544363eec513d', 0.8395819982490089),
 ('66635739c2f544363eee90b5', 0.8390102340070982),
 ('6663569ac2f544363eec2d7d', 0.8321176722540172),
 ('6663569bc2f544363eec3692', 0.8273647547565525),
 ('6663569ac2f544363eec2d7c', 0.8126824013686896),
 ('6663569ec2f544363eec6808', 0.8047962280147691),
 ('6663569dc2f544363eec5f6b', 0.772065280454064),
 ('6663569ec2f544363eec719f', 0.7572072647332523),
 ('6663569fc2f544363eec817d', 0.7387617729115047),
 ('6663569fc2f544363eec7bb6', 0.7329597829456506),
 ('6663568cc2f544363eec07a5', 0.7

In [12]:
with open("output/log.txt", 'r') as file:
    lines = file.readlines()

In [13]:
timestamps_list = [list(map(float, line.strip().split(','))) for line in lines]

In [14]:
durations = {
    "bm25_duration": [],
    "bi_encoder_duration": [],
    "cross_encoder_duration": [],
    "ensemble_duration": [],
    "fetch_answer_duration": [],
    "system_duration": []
}

for timestamps in timestamps_list:
    durations["bm25_duration"].append(timestamps[0])
    durations["bi_encoder_duration"].append(timestamps[1] - timestamps[0])
    durations["cross_encoder_duration"].append(timestamps[2] - timestamps[1])
    durations["ensemble_duration"].append(timestamps[3] - timestamps[2])
    durations["fetch_answer_duration"].append(timestamps[4] - timestamps[3])
    durations["system_duration"].append(timestamps[4])

average_durations = {key: sum(values)/len(values) for key, values in durations.items()}

In [17]:
for process, avg_duration in average_durations.items():
    print(f"Average {process.replace('_', ' ')}: {avg_duration:.6f} seconds")

Average bm25 duration: 0.082932 seconds
Average bi encoder duration: 0.071993 seconds
Average cross encoder duration: 0.241936 seconds
Average ensemble duration: 0.000033 seconds
Average fetch answer duration: 0.033574 seconds
Average system duration: 0.430469 seconds
